题目：

为22.news.dat这篇文档中的长篇文章进行主题训练，使用LDA模型，并计算各部分用时(基本方法和01一样，就是文档大了许多)

## 1.导入库

In [56]:
import numpy as np
from gensim import corpora, models, similarities
from pprint import pprint
import time

## 2.数据预处理

In [42]:
# 定义一个停止词函数
def load_stopword():
    f_stop = open('22.stopword.txt')
    sw = [line.strip() for line in f_stop]
    f_stop.close()
    return sw

读入文本，并进行停止词的去除工作

In [43]:
t_start = time.time()

stop_words = load_stopword()  # 初始化停止词列表

# 读入文本
print('开始读入语料数据 -- ')
f = open('22.news.dat','rb')
texts = [[word for word in line.strip().lower().split() if word not in stop_words] for line in f] # 去除停止词
# texts = [line.strip().split() for line in f]
print('读入语料数据完成，用时%.3f秒' % (time.time() - t_start))

f.close()
M = len(texts)
print('文本数目：%d个' % M)
# pprint(texts)

开始读入语料数据 -- 
读入语料数据完成，用时11.921秒
文本数目：2043个


进行 TF-IDF 文档向量化处理

In [44]:
# 建立字典用于TF-IDF
dictionary = corpora.Dictionary(texts)
V = len(dictionary)

# 计算文本向量
corpus = [dictionary.doc2bow(text) for text in texts]

print('正在计算文档TF-IDF --')
t_start = time.time()
corpus_tfidf = models.TfidfModel(corpus)[corpus]
print('建立文档TF-IDF完成，用时%.3f秒' % (time.time() - t_start))

正在计算文档TF-IDF --
建立文档TF-IDF完成，用时0.227秒


## 3.进行模型训练

In [45]:
# 进行LDA模型拟合推断
num_topics = 30  # 主题设为30个
t_start = time.time()

# 进行模型训练 将之前处理好的 corpus_tfidf 文件喂给模型，主题30个，超参数为0.01
lda = models.LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary,
                        alpha=0.01, eta=0.01, minimum_probability=0.001,
                        update_every = 1, chunksize = 100, passes = 1)
print('LDA模型完成，训练时间为\t%.3f秒' % (time.time() - t_start))

#  所有文档的主题
# doc_topic = [a for a in lda[corpus_tfidf]]
# print 'Document-Topic:\n'
# pprint(doc_topic)

LDA模型完成，训练时间为	10.797秒


## 4.观察训练完成后的结果

每个文档的主题分布

In [48]:
# 随机打印某10个文档的主题
num_show_topic = 10  # 每个文档显示前几个主题
print('随机10个文档的主题分布：')
doc_topics = lda.get_document_topics(corpus_tfidf)  # 所有文档的主题分布
idx = np.arange(M)
np.random.shuffle(idx)  # 将文档打乱
idx = idx[:10]

for i in idx:  # 遍历选出的10个文档
    topic = np.array(doc_topics[i])  # 用numpy的形式，方便操作
    topic_distribute = np.array(topic[:, 1])
    # print topic_distribute
    topic_idx = topic_distribute.argsort()[:-num_show_topic-1:-1]
    print('第%d个文档的前%d个主题：' % (i, num_show_topic)), topic_idx
    print(topic_distribute[topic_idx])

随机10个文档的主题分布：
第550个文档的前10个主题：
[0.38123742 0.28194046 0.13068601 0.09316909 0.00434489 0.00434489
 0.00434489 0.00434489 0.00434489 0.00434489]
第1729个文档的前10个主题：
[0.53179723 0.2066005  0.08706684 0.05550412 0.04830942 0.00282888
 0.00282888 0.00282888 0.00282888 0.00282888]
第419个文档的前10个主题：
[0.53000224 0.1510151  0.13183205 0.08676627 0.0486549  0.00206918
 0.00206918 0.00206918 0.00206918 0.00206918]
第714个文档的前10个主题：
[0.44119251 0.25713351 0.09403287 0.07982886 0.06638387 0.00245713
 0.00245713 0.00245713 0.00245713 0.00245713]
第1818个文档的前10个主题：
[0.42926741 0.27886125 0.16360036 0.05230379 0.00292181 0.00292181
 0.00292181 0.00292181 0.00292181 0.00292181]
第260个文档的前10个主题：
[0.59973681 0.21033934 0.006783   0.00678299 0.00678299 0.00678299
 0.00678299 0.00678299 0.00678299 0.00678299]
第1786个文档的前10个主题：
[0.50740391 0.16529153 0.13587341 0.07273821 0.00456512 0.00456512
 0.00456512 0.00456512 0.00456512 0.00456512]
第927个文档的前10个主题：
[0.64895588 0.13678253 0.09498405 0.00441768 0.00441768 0.004417

每个主题的词分布

In [47]:
# 每个主题的词分布（共30个主题）
num_show_term = 7   # 每个主题显示几个词

for topic_id in range(num_topics):
    print('主题%d:' % topic_id)
    term_distribute_all = lda.get_topic_terms(topicid=topic_id)
    term_distribute = term_distribute_all[:num_show_term]
    term_distribute = np.array(term_distribute)
    term_id = term_distribute[:, 0].astype(np.int)
    for t in term_id:
        print(dictionary.id2token[t])
        # print '\n概率：\t', term_distribute[:, 1]

主题0:
公司
女士
旅游
创新
村
合作
规划
主题1:
中方
答
航班
美方
联合国
有何
谈判
主题2:
韩国
广岛
奥巴马
巴西
英国
原子弹
加拿大
主题3:
算你狠
白鹭洲
翻篇
缥缈
练功
约法三章
老家伙
主题4:
仲裁
领土
案
司法
轿车
绑架
法官
主题5:
德国
郑捷
欧洲
捷运
死刑
台北
村民
主题6:
李某
陈某
考生
招生
宋某
高考
同比
主题7:
督察
环保
督察组
列车
河北
铁路
习近平
主题8:
算你狠
白鹭洲
翻篇
缥缈
练功
约法三章
老家伙
主题9:
司机
检察院
公交车
公交
车厢
乘客
车辆
主题10:
雷洋
炸弹
人大
is
爆炸
伊拉克
汽车
主题11:
日本
南海
菲律宾
解决
叙利亚
共和党
女生
主题12:
滨海
女子
行凶
捅
保安
小球
客服
主题13:
朝鲜
金正恩
地震
朝鲜劳动党
度
东经
震源
主题14:
签证
用户
高速
法国
百度
欧元
公司
主题15:
幼儿园
仪征市
园区
学生家长
搬迁
实验
新
主题16:
灾害
福建
搜救
救援
泰宁县
泰宁
泥石流
主题17:
特朗普
总统
民警
医生
着
执法
这些
主题18:
天气
大风
受灾
广西
紧急
遇难
住宅
主题19:
她
女子
视频
日本
治疗
当地
越南
主题20:
驾驶
交警
某某
摩托车
房主
行驶
违建
主题21:
先生
保健品
女性
自由
男性
旅行社
旅客
主题22:
蔡
大陆
英文
台湾
两岸
马英九
民进党
主题23:
任
党委
纪律
巡视
纪委
市委
纪委书记
主题24:
医院
中国
警方
我
美国
他
了
主题25:
阅读
读书
俄罗斯
书籍
电子
意思
普京
主题26:
被告人
同志
人民法院
法庭
中级
行贿
指控
主题27:
印尼
充气
帕廷
天使
娃娃
神灵
捡
主题28:
算你狠
白鹭洲
翻篇
缥缈
练功
约法三章
老家伙
主题29:
细胞
临床
有偿
二院
生物
疗法
患者
